In [5]:
import pandas as pd
import numpy as np
import xarray as xr
import os, sys
import glob
import pdb
from tqdm.notebook import tqdm, tnrange
tqdm().pandas()

Join multiple files into one using NCO utilities.
[here](http://forum.marine.copernicus.eu/discussion/205/how-to-mergeconcatenate-cmems-netcdf-files-tips/p1)

The following routine concats files without the need for NCO, but can be slow if there are many files.

In [6]:
files = glob.glob('/storage/ShenResults/*.csv')

In [49]:
def save_as_netcdf(ds, filename):
    filename = filename.replace('.csv', '.nc')
    comp = dict(zlib=True, complevel=9) # compresss level set to max
    encoding = {var: comp for var in ds.data_vars}
    ds.to_netcdf(filename, encoding=encoding)
    
# convert csv to nc. Need to leave out index for now.
for file in tqdm(files): # each file is 
    try:
        df = pd.read_csv(file, index_col=0)
        df = df.drop_duplicates(keep='first')
        df = df[['lat', 'long', 'mean', 'tdx', 'pres']]
        df = df.set_index(['lat', 'long', 'pres', 'tdx'])
        ds = df.to_xarray()
        ds = ds.set_coords(['lat', 'long', 'tdx', 'pres']).copy()
        #ds = xr.Dataset.from_dataframe(df)
        save_as_netcdf(ds, file)
    except Exception as err:
        pdb.set_trace()
        print(err)
        continue

In [40]:
file

'/storage/ShenResults/mTempData_1875.0.csv'

In [81]:
df = pd.read_csv(file, index_col=0)
df = df.drop_duplicates(keep='first')
df.head()

,lat,long,mean,std,dof,startDate,endDate,tdx,pres
0,-69.25,-3.25,0.052000,0.019799,3,2004-01-01,2004-01-05,0,1875.0
1,-68.00,-12.75,-0.020333,0.025329,3,2004-01-01,2004-01-05,0,1875.0
2,-64.50,15.75,0.202000,0.016000,2,2004-01-01,2004-01-05,0,1875.0
3,-64.00,-11.75,-0.080000,0.000000,1,2004-01-01,2004-01-05,0,1875.0
4,-64.00,-8.75,-0.102667,0.018373,3,2004-01-01,2004-01-05,0,1875.0


In [ ]:
x = np.arange(-180, 180, .25)
y = np.arange(-90, 90, .25)
xv, yv = np.meshgrid(x, y, sparse=False) 
cdf = pd.DataFrame({'lat': yv.flatten(), 'long': xv.flatten()})
result = pd.concat([df, cdf], axis=1, keys=['lat', 'long'], sort=False)

array([-90.  , -90.  , -90.  , ...,  89.75,  89.75,  89.75])

In [61]:
df = df[['lat', 'long', 'mean', 'tdx', 'pres']]

In [62]:
df = df.set_index(['lat', 'long'])

In [63]:
ds = df.to_xarray()

ValueError: cannot handle a non-unique multi-index!

In [56]:
#ds2 = ds.set_coords(['lat', 'long', 'tdx', 'pres'])

In [57]:
ds2.expand_dims(['lat', 'long', 'tdx', 'pres'])

ValueError: lat already exists as coordinate or variable name.

In [32]:
ds = xr.Dataset({'a': xr.Variable(('lon', 'lat'), np.random.random((4, 5))), 'b': xr.Variable(('lon', 'lat'),np.random.random((4, 5))), 'longitude':
    ...:  xr.Variable('lon', [1, 2, 3, 4]), 'latitude': xr.Variable('lat', [1, 2, 3, 4, 5])})

In [33]:
ds

<xarray.Dataset>
Dimensions:    (lat: 5, lon: 4)
Dimensions without coordinates: lat, lon
Data variables:
    a          (lon, lat) float64 0.8481 0.7451 0.7074 ... 0.2091 0.8746 0.3428
    b          (lon, lat) float64 0.1059 0.8322 0.8594 ... 0.9245 0.907 0.2402
    longitude  (lon) int64 1 2 3 4
    latitude   (lat) int64 1 2 3 4 5

In [36]:
ds = ds.rename({'latitude': 'lat', 'longitude': 'lon'}).set_coords(['lon', 'lat'])

In [37]:
ds

<xarray.Dataset>
Dimensions:  (lat: 5, lon: 4)
Coordinates:
  * lon      (lon) int64 1 2 3 4
  * lat      (lat) int64 1 2 3 4 5
Data variables:
    a        (lon, lat) float64 0.8481 0.7451 0.7074 ... 0.2091 0.8746 0.3428
    b        (lon, lat) float64 0.1059 0.8322 0.8594 ... 0.9245 0.907 0.2402

In [32]:
dataSets = []
nfiles = glob.glob('/storage/ShenResults/*.nc')
for file in tqdm(nfiles):
    ds = xr.load_dataset(file)
    dataSets.append(ds)

In [ ]:
combined = xr.concat(dataSets, dim='pres') # can concat along pressure dimension

In [ ]:
save_as_netcdf(combined, 'meanTemp.nc')

In [6]:
file

'/storage/ShenResults/mTempData_3.75.csv'

In [6]:
comp = dict(zlib=True, complevel=9) # compresss level set to max
encoding = {var: comp for var in combined.data_vars}
combined.to_netcdf('mTempData.nc', encoding=encoding)

# Check if file was created properly

In [4]:
ds = xr.open_dataset('mTempData.nc')

In [5]:
ds

<xarray.Dataset>
Dimensions:    (lat: 552, long: 1233, pres: 26, tdx: 2)
Coordinates:
  * lat        (lat) float64 -89.0 -74.25 -73.75 -73.5 ... 72.0 74.5 75.25 78.25
  * long       (long) float64 -179.5 -179.2 -179.0 -178.8 ... 179.2 179.5 179.8
  * tdx        (tdx) int64 1093 1094
  * pres       (pres) float64 3.75 9.75 16.0 ... 1.45e+03 1.625e+03 1.875e+03
Data variables:
    mean       (lat, long, tdx, pres) float64 ...
    std        (lat, long, tdx, pres) float64 ...
    dof        (lat, long, tdx, pres) float64 ...
    startDate  (lat, long, tdx, pres) object ...
    endDate    (lat, long, tdx, pres) object ...

In [21]:
len(ds.lat.values)

552

In [22]:
len(ds.long.values)

1233

In [7]:
df = ds.to_dataframe()

KeyboardInterrupt: 

In [6]:
df = df.dropna(axis=0)

NameError: name 'df' is not defined

In [43]:
df.head()

mean       std   dof   startDate     endDate
lat   long pres  tdx                                                   
-89.0 -0.0 3.75  1094 -1.471143  0.004502  14.0  2018-12-27  2018-12-31
           2.25  1094 -1.470500  0.003775   8.0  2018-12-27  2018-12-31
           10.00 1094  0.823216  1.202438  37.0  2018-12-27  2018-12-31
           4.00  1094 -1.277733  0.158675  15.0  2018-12-27  2018-12-31
           5.00  1094 -0.959263  0.280545  19.0  2018-12-27  2018-12-31